<a href="https://colab.research.google.com/github/fatemafaria142/Multimodal-Language-Models/blob/main/Image_Caption_Generation_using_imp_v1_3B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers -q
!pip install -q pillow accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.2 MB/s eta 0:00:00


In [10]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image

In [7]:
torch.set_default_device("cuda")

### **Model and Tokenizer Link:** https://huggingface.co/MILVLG/imp-v1-3b

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    "MILVLG/imp-v1-3b",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

configuration_imp.py:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/MILVLG/imp-v1-3b:
- configuration_imp.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_imp.py:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

...

model-00005-of-00007.safetensors:   0%|          | 0.00/997M [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/370M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    "MILVLG/imp-v1-3b",
    trust_remote_code=True
)

tokenizer_config.json:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Image Analysis and Response Generation with AI Assistant with Prompt 1**

In [22]:
def generate_output_for_images(image_paths):
    for image_path in image_paths:
        # Load image
        image = Image.open(image_path)

        # Prepare text and image for the model
        text = f"A chat between a curious user and an artificial intelligence assistant. The AI assistant crafts a creative caption for the image.\nUSER: <image>\nGenerate an awesome caption. ASSISTANT:"
        input_ids = tokenizer(text, return_tensors="pt").input_ids
        image_tensor = model.image_preprocess(image)

        # Generate output
        output_ids = model.generate(
            input_ids,
            max_new_tokens=100,
            images=image_tensor,
            use_cache=True
        )[0]

        # Print the decoded output
        decoded_output = tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip()
        print(f"Output for {image_path}:\n Image Caption: {decoded_output}\n")


In [20]:
# Directory containing images
image_directory = '/content/drive/MyDrive/images_for_model'

# List all files in the directory
import os
image_paths = [os.path.join(image_directory, file) for file in os.listdir(image_directory) if file.endswith(('.jpg', '.JPG','.png', '.jpeg'))]

In [23]:
# Print the list of image paths
print("Image Paths:")
print(image_paths)

Image Paths:
['/content/drive/MyDrive/images_for_model/image_2.jpg', '/content/drive/MyDrive/images_for_model/image_4.jpg', '/content/drive/MyDrive/images_for_model/image_3.jpg', '/content/drive/MyDrive/images_for_model/image_5.JPG', '/content/drive/MyDrive/images_for_model/image_1.JPG']


In [24]:
# Generate output for each image
generate_output_for_images(image_paths)

Output for /content/drive/MyDrive/images_for_model/image_2.jpg:
 Image Caption: A woman wearing a blue sari and pink bangles poses in front of a tree.

Output for /content/drive/MyDrive/images_for_model/image_4.jpg:
 Image Caption: A group of women wearing colorful saris pose for a photo.

Output for /content/drive/MyDrive/images_for_model/image_3.jpg:
 Image Caption: A woman wearing a black and yellow sari sits on a bench, posing for a photo.

Output for /content/drive/MyDrive/images_for_model/image_5.JPG:
 Image Caption: A woman in a floral dress and scarf is sitting on the grass, enjoying the sunshine.

Output for /content/drive/MyDrive/images_for_model/image_1.JPG:
 Image Caption: Two women posing for a photo, one wearing a red T-shirt and the other in a white dress.



### **Image Analysis and Response Generation with AI Assistant with Prompt 2**

In [28]:
def generate_output_for_images_poetic_caption(image_paths):
    for image_path in image_paths:
        # Load image
        image = Image.open(image_path)

        # Prepare text and image for the model
        text = f"A chat between a curious user and an artificial intelligence assistant. The AI assistant crafts a lyrical masterpiece caption for the image. \nUSER: <image>\nGenerate a mesmerizing poetic caption. ASSISTANT:"
        input_ids = tokenizer(text, return_tensors="pt").input_ids
        image_tensor = model.image_preprocess(image)

        # Generate output
        output_ids = model.generate(
            input_ids,
            max_new_tokens=100,
            images=image_tensor,
            use_cache=True
        )[0]

        # Print the decoded output
        decoded_output = tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip()
        print(f"Output for {image_path}:\n Image Caption: {decoded_output}\n")


In [29]:
# Generate output for each image
generate_output_for_images_poetic_caption(image_paths)

Output for /content/drive/MyDrive/images_for_model/image_2.jpg:
 Image Caption: A woman stands tall, her long hair cascading down her back, adorned with a blue saree and a gold necklace. She is surrounded by nature, with a tree in the background, and a potted plant nearby. Her beauty shines through as she poses for the camera, capturing a moment of elegance and grace.

Output for /content/drive/MyDrive/images_for_model/image_4.jpg:
 Image Caption: A group of women, dressed in vibrant saris, stand together in a picturesque setting. Their smiles radiate warmth and joy, as they pose for a memorable moment. The scene captures the essence of friendship and celebration, with each woman showcasing her unique beauty and style. The colorful attire and the camaraderie among the group create a captivating and enchanting atmosphere.

Output for /content/drive/MyDrive/images_for_model/image_3.jpg:
 Image Caption: A woman in a vibrant black and yellow sari sits gracefully, her eyes sparkling with co